In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np
from contextlib import redirect_stdout
import time

Building the topology and selecting the atoms in each chain

In [ ]:
work_dir = '/home/eva/Documents/gbsa-sims/cv-sims/monomer_restraints/cvmon-1/tet_L24E/GLU/'

top_dir = '/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/'

In [ ]:
prot = md.load_pdb(top_dir+'6q5i_GLU_cv.pdb')
topology = prot.topology

g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]
g3 = [atom.index for atom in topology.chain(2).atoms]
g4 = [atom.index for atom in topology.chain(3).atoms]

In [ ]:
topology

First finding out which atoms participate in phi and psi dihedral angles

Loading the inpcrd file instead of a trajectory, as we are only interested in which atoms participate in the dihedrals

In [ ]:
coord = md.load(top_dir+'6q5i_GLU.inpcrd', top = top_dir+'6q5i_GLU.prmtop')

In [ ]:
phi_array = md.compute_phi(coord)
psi_array = md.compute_psi(coord)

In [ ]:
phi_atoms = phi_array[0].tolist()
psi_atoms = psi_array[0].tolist()

Select residues 5-25 in each chain to pose restraints

In [ ]:
#selecting residues 5-25, needs to be implemented later

In [ ]:
start_time = time.time()

Modified energy expression with individual harmonic potentials.
Note that there is one CV (r0)

In [ ]:
prmtop = AmberPrmtopFile(top_dir+'6q5i_GLU.prmtop')
inpcrd = AmberInpcrdFile(top_dir+'6q5i_GLU.inpcrd')
pdb = PDBFile(top_dir+'6q5i_GLU_cv.pdb')

system = prmtop.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, hydrogenMass=1.5*amu, 
                             implicitSolvent=OBC2)

expr = '0.5*k*(distance(g1,g2)-r0)^2 + 0.5*k*(distance(g1,g3)-distance(g1,g2))^2 + 0.5*k*(distance(g1,g4)-distance(g1,g2))^2 + 0.5*k*(distance(g2,g3)-distance(g1,g2))^2 + 0.5*k*(distance(g2,g4)-distance(g1,g2))^2 + 0.5*k*(distance(g3,g4)-distance(g1,g2))^2'

force = openmm.CustomCentroidBondForce(4, expr)
force.addGlobalParameter('k', 100.0*kilojoules_per_mole/nanometers**2)
force.addGlobalParameter('r0', 50*nanometers)
force.addGroup(g1)
force.addGroup(g2)
force.addGroup(g3)
force.addGroup(g4)
force.addBond([0,1,2,3])
force.setForceGroup(1)
system.addForce(force)

#average alpha helix phi and psi dihedrals
phi0 = (-57.0*np.pi)/180.0
psi0 = (-47.0*np.pi)/180.0

phiforce = CustomTorsionForce('k_dihed*(1+ cos(theta-phi0))')
phiforce.addGlobalParameter('k_dihed', 10.0*kilojoules_per_mole)
phiforce.addGlobalParameter('phi0', phi0)

psiforce = CustomTorsionForce('k_dihed*(1+ cos(theta-psi0))')
psiforce.addGlobalParameter('k_dihed', 10.0*kilojoules_per_mole)
psiforce.addGlobalParameter('psi0', psi0)


for i in range(len(phi_atoms)):
    phiforce.addTorsion(phi_atoms[i][0], phi_atoms[i][1], phi_atoms[i][2], phi_atoms[i][3])
    psiforce.addTorsion(psi_atoms[i][0], psi_atoms[i][1], psi_atoms[i][2], psi_atoms[i][3])
    
phiforce.setForceGroup(2)
psiforce.setForceGroup(3)
system.addForce(phiforce)
system.addForce(psiforce)

#adding a ghost force whose energy numerical value is equal to the distance between the monomers
#so equal to the collective variable
#CV force easier to query than other custom forces
dist = 'distance(g1,g2)+distance(g1,g3)+distance(g1,g4)+distance(g2,g3)+distance(g2,g4)+distance(g3,g4)'
ghost_force = openmm.CustomCentroidBondForce(4, dist)
ghost_force.addGroup(g1)
ghost_force.addGroup(g2)
ghost_force.addGroup(g3)
ghost_force.addGroup(g4)
ghost_force.addBond([0,1,2,3])
#ghost_force.setForceGroup(4)
#system.addForce(ghost_force)

cvforce = openmm.CustomCVForce('r')
cvforce.addCollectiveVariable('r', ghost_force)
cvforce.setForceGroup(4) #have to exclude force group 4 from the integrator
system.addForce(cvforce)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)
integrator.setIntegrationForceGroups(set(range(4))) #excluding ghost force/CV force from the integration

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)

simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter(work_dir+'test-output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))

print(cvforce.getCollectiveVariableValues(simulation.context)) #get CV value for the input structure

simulation.step(100000)

print ("simulation time:", time.time() - start_time, "s")

In [ ]:
traj = md.load(work_dir+'test-output.dcd', top = top_dir+'6q5i_GLU.prmtop')

forces = []
cv_forces = []

for crd in traj.xyz: #getting atom coordinates in each frame to make the simulation context
    simulation.context.setPositions(crd)
    cv_forces.append(cvforce.getCollectiveVariableValues(simulation.context)) #get CV per frame
    forces.append(simulation.context.getState(getForces=True, groups={1}).getForces(asNumpy=True)) 
    #query custom centroid bond force only

In [ ]:
cvforce_list =[]
for i in cv_forces:
    cvforce_list.append(i[0])

with open(work_dir+'cv-values.txt', 'w') as out:
        with redirect_stdout(out):
            for frc in cvforce_list:
                print(frc)

In [ ]:
g1_positions=[]
g2_positions=[]
g3_positions=[]
g4_positions=[]

#get x,y,z coordinates for each individual chain
for crd in traj.xyz:
    g1_positions.append(crd[g1[0]:g1[-1]+1]) 
    g2_positions.append(crd[g2[0]:g2[-1]+1])
    g3_positions.append(crd[g3[0]:g3[-1]+1])
    g4_positions.append(crd[g4[0]:g4[-1]+1])

In [ ]:
#forces = np.vstack(forces)

In [ ]:
#algebraic expression for calculating CV per frame - comparison with CVs computed from simulation context

cv_linalg = []

for idx in range(len(traj.xyz)):
    dist12 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g2_positions[idx]), axis=0))
    dist13 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g3_positions[idx]), axis=0))
    dist14 = np.linalg.norm(np.mean(np.asarray(g1_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    dist23 = np.linalg.norm(np.mean(np.asarray(g2_positions[idx]),axis=0) - np.mean(np.asarray(g3_positions[idx]), axis=0))
    dist24 = np.linalg.norm(np.mean(np.asarray(g2_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    dist34 = np.linalg.norm(np.mean(np.asarray(g3_positions[idx]),axis=0) - np.mean(np.asarray(g4_positions[idx]), axis=0))
    
    cv = dist12+dist13+dist14+dist23+dist24+dist34
    cv_linalg.append(cv)

In [ ]:
with open(work_dir+'cv-linalg.txt', 'w') as out:
        with redirect_stdout(out):
                for j in cv_linalg:
                    print(j)